In [1]:
# A script to query a chatbot using a concise prompt template via LangChain and Ollama.

import argparse  # For handling command-line arguments
from langchain.prompts import ChatPromptTemplate  # For formatting prompts
from langchain_community.llms.ollama import Ollama  # For interacting with Ollama models
import sys  # For checking script or interactive mode

# Template to strictly generate concise responses.
PROMPT_TEMPLATE = """
Strictly answer the question in 2 sentences.
Answer the question: {question}
"""

def main(query_text):
    
    # Main function to process the user query and fetch chatbot response.
    
    query_chatbot(query_text)

def query_chatbot(query_text: str):
    """
    Queries the chatbot with the given text and returns the response.

    Args:
        query_text (str): The user's question/query.

    Returns:
        str: The chatbot's response.
    """
    # Format the prompt using the provided template
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(question=query_text)
    
    # Initialize the model
    model = Ollama(model="vicuna:13b-q8_0")  # Update model name if needed
    response_text = model.invoke(prompt)

    # Format and display the response
    formatted_response = f"Response: {response_text}"
    print(formatted_response)
    return response_text

if __name__ == "__main__":
    if sys.stdin.isatty():  # Description: Script mode to accept command-line arguments.
        parser = argparse.ArgumentParser()
        parser.add_argument("query_text", type=str, help="The query text.")
        args = parser.parse_args()
        main(args.query_text)
    else:  # Description: Interactive mode to accept user input via terminal.
        query_text = input("Enter your query: ")
        main(query_text)


Enter your query: What is bitcoin?


/tmp/ipykernel_928/1081779223.py:35: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="vicuna:13b-q8_0")  # Update model name if needed


Response: Bitcoin is a decentralized digital currency that uses cryptography for security and is not controlled by any government or financial institution. It was created in 2009 by an unknown person or group using the name Satoshi Nakamoto.


In [2]:
# Script to evaluate a NON-RAG model using BLEU, ROUGE-L, and BERTScore metrics on a Q&A dataset.

import pandas as pd  # For data manipulation
import nltk  # For natural language processing
from rouge_score import rouge_scorer  # For calculating ROUGE scores
from bert_score import score as bert_score  # For calculating BERTScore
from nltk.translate.bleu_score import sentence_bleu  # For calculating BLEU scores
import os  # For system operations

# Ensure NLTK models are downloaded
nltk.download('punkt')

# Calculates BLEU score for reference and candidate responses.
def calculate_bleu(reference, candidate):
    reference_tokens = [nltk.word_tokenize(reference)]
    candidate_tokens = nltk.word_tokenize(candidate)
    return sentence_bleu(reference_tokens, candidate_tokens)

# Calculates ROUGE-L score for reference and candidate responses.
def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores['rougeL'].fmeasure

# Calculates BERTScore for reference and candidate responses.
def calculate_bertscore(references, candidates):
    P, R, F1 = bert_score(candidates, references, lang='en', rescale_with_baseline=True)
    return F1.mean().item()

# Weights for final score calculation
weight_bleu = 0.3
weight_rouge = 0.3
weight_bertscore = 0.7

# Load the dataset from a CSV file.
excel_path = 'QandA_Short.csv'  # Adjust the file name as needed
df = pd.read_csv(excel_path)

# Define column names based on the dataset structure
question_column = 'Questions'         # Column with questions
gold_answer_column = 'Answers'        # Column with reference answers

# Lists to store evaluation metrics and model responses
bleu_scores = []
rouge_scores = []
bert_scores = []
final_scores = []
model_responses = []

# Loop through each row in the DataFrame to evaluate responses.
for i, row in df.iterrows():
    question = row[question_column]
    gold_answer = row[gold_answer_column]

    # Generate the model's response
    model_response = query_chatbot(question) or "No response"  # Default to "No response" on failure
    model_response = model_response.strip()  # Clean the response
    print(f"Question: {question}\nModel Response: {model_response}\n")
    print("Q no.", i)
    model_responses.append(model_response)

    # Calculate evaluation metrics
    bleu = calculate_bleu(gold_answer, model_response)
    rouge = calculate_rouge(gold_answer, model_response)
    bertscore = calculate_bertscore([gold_answer], [model_response])

    # Compute the final weighted score
    final_score = (
        weight_rouge * rouge +
        weight_bertscore * bertscore
    )
    
    # Append metrics to respective lists
    bleu_scores.append(bleu)
    rouge_scores.append(rouge)
    bert_scores.append(bertscore)
    final_scores.append(final_score)

# Play a sound notification upon completion
os.system('afplay /System/Library/Sounds/Sosumi.aiff')

# Add evaluation metrics to the DataFrame.
df['Model Response'] = model_responses
df['ROUGE-L Score'] = rouge_scores
df['BERTScore'] = bert_scores
df['Final Score'] = final_scores

# Display the first few rows of the DataFrame for verification
print(df.head())

# Save the updated DataFrame with evaluation metrics to a CSV file.
output_path = 'output_file_NonRAG_vicuna_short.csv'
df.to_csv(output_path, index=False)
print(f"Metrics have been saved to {output_path}")


[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Response: Cryptocurrency is a digital or virtual currency that uses encryption techniques to control the creation of new units and verify transactions, operating independently of a central bank. It allows for decentralized transactions and typically functions as a medium of exchange, a store of value, or a unit of account.
Question: What is cryptocurrency?
Model Response: Cryptocurrency is a digital or virtual currency that uses encryption techniques to control the creation of new units and verify transactions, operating independently of a central bank. It allows for decentralized transactions and typically functions as a medium of exchange, a store of value, or a unit of account.

Q no. 0


2024-11-25 03:16:14.389104: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 03:16:14.400496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732504574.414933     928 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732504574.419230     928 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 03:16:14.434974: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Response: Cryptocurrencies use encryption techniques to secure financial transactions and control the creation of new units, operating independently of a central bank.
Question: How do cryptocurrencies work?
Model Response: Cryptocurrencies use encryption techniques to secure financial transactions and control the creation of new units, operating independently of a central bank.

Q no. 1


/home/student/venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/student/venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/student/venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunc

...

Response: An Initial Exchange Offering (IEO) is a fundraising mechanism where a startup sells its tokens or cryptocurrency on a cryptocurrency exchange instead of conducting an ICO. The exchange acts as a intermediary and guarantees the project's legitimacy, which helps attract investors who may be wary of scams.
Question: What is an Initial Exchange Offering (IEO)?
Model Response: An Initial Exchange Offering (IEO) is a fundraising mechanism where a startup sells its tokens or cryptocurrency on a cryptocurrency exchange instead of conducting an ICO. The exchange acts as a intermediary and guarantees the project's legitimacy, which helps attract investors who may be wary of scams.

Q no. 94


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Response: To participate in an IEO (Initial Exchange Offering), you need to follow these steps:

1. Familiarize yourself with the project and its team.
2. Choose a supported exchange platform that's hosting the IEO.
3. Acquire the relevant cryptocurrency on the chosen exchange before or during the IEO launch.
4. Follow the exchange's instructions to participate in the IEO, which may involve creating an account and depositing funds.
Question: How do I participate in an IEO?
Model Response: To participate in an IEO (Initial Exchange Offering), you need to follow these steps:

1. Familiarize yourself with the project and its team.
2. Choose a supported exchange platform that's hosting the IEO.
3. Acquire the relevant cryptocurrency on the chosen exchange before or during the IEO launch.
4. Follow the exchange's instructions to participate in the IEO, which may involve creating an account and depositing funds.

Q no. 95


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
sh: 1: afplay: not found


                         Questions  \
0          What is cryptocurrency?   
1    How do cryptocurrencies work?   
2   What is blockchain technology?   
3                 What is Bitcoin?   
4                What is Ethereum?   

                                             Answers  \
0  Cryptocurrency is a digital or virtual currenc...   
1  Cryptocurrencies function through a decentrali...   
2  Blockchain technology is a distributed ledger ...   
3  Bitcoin is the first decentralized cryptocurre...   
4  Ethereum is a decentralized platform that allo...   

                                      Model Response  ROUGE-L Score  \
0  Cryptocurrency is a digital or virtual currenc...       0.337349   
1  Cryptocurrencies use encryption techniques to ...       0.156863   
2  Blockchain technology is a decentralized digit...       0.517241   
3  Bitcoin is a decentralized digital currency th...       0.376812   
4  Ethereum is a decentralized platform that enab...       0.325581   

   BERT